# Sessão 6 – Pipeline de Múltiplas Etapas (Planejar → Executar → Refinar)

Utiliza funções de roteamento para selecionar modelos em cada etapa e, em seguida, refina a resposta final.


### Explicação: Instalação de Dependências
Instala `foundry-local-sdk` e `openai`, necessários para chamadas de chat por etapa. Seguro para executar novamente.


# Cenário
Pipeline de modelo em múltiplas etapas que: (1) Planeja uma tarefa em etapas discretas, (2) Executa cada etapa com seleção de modelo baseada em intenção, (3) Refina uma resposta final sintetizada. Demonstra a integração de capacidades heterogêneas de SLM para eficiência, mantendo a qualidade.


In [13]:
!pip install -q foundry-local-sdk openai

### Explicação: Importações Principais
Importa regex para detecção de intenções, Foundry Local manager para anexação por alias, e o cliente OpenAI para completude de chats.


In [14]:
import re
from foundry_local import FoundryLocalManager
from openai import OpenAI

### Explicação: Catálogo de Capacidades e Regras
Define um catálogo baseado em capacidades e regras regex usadas para mapear o texto das etapas para categorias de intenção (código, resumir, classificação, geral). Valores de prioridade mais baixos vencem em casos de empate quando vários modelos suportam uma intenção.


In [15]:
CATALOG = {
 'phi-4-mini': {'capabilities':['general','summarize'],'priority':2},
 'qwen2.5-coder-7b': {'capabilities':['code','refactor'],'priority':1},
 'qwen2.5-0.5b': {'capabilities':['classification','fast'],'priority':3},
}
RULES = [
 (re.compile('code|refactor|function', re.I), 'code'),
 (re.compile('summari|abstract|tl;dr', re.I), 'summarize'),
 (re.compile('classif|category|label', re.I), 'classification'),
]

### Explicação: Intenção, Seleção de Modelo e Auxiliar de Chat
Fornece:
- `detect_intent` para classificação baseada em regex.
- `pick_model` para escolher o melhor alias com base em capacidade + prioridade.
- `chat`, um wrapper prático que instancia um cliente por alias e retorna uma resposta de turno único.


In [16]:
def detect_intent(text: str):
    """Return an intent label based on simple regex rules; falls back to 'general'."""
    for pat, intent in RULES:
        if pat.search(text):
            return intent
    return 'general'

def pick_model(intent: str) -> str:
    """Pick the best model for an intent using capability match first, then priority."""
    ranked = []
    for name, meta in CATALOG.items():
        ranked.append((name, intent in meta['capabilities'], meta['priority']))
    # Sort: capability match (True first), then lower priority value
    ranked.sort(key=lambda t: (not t[1], t[2]))
    return ranked[0][0]

def chat(alias: str, content: str, temp: float = 0.4) -> str:
    """Simple helper to send a single user message to a Foundry Local model via OpenAI client."""
    m = FoundryLocalManager(alias)
    client = OpenAI(base_url=m.endpoint, api_key=m.api_key or 'not-needed')
    mid = m.get_model_info(alias).id
    resp = client.chat.completions.create(
        model=mid,
        messages=[{'role': 'user', 'content': content}],
        max_tokens=180,
        temperature=temp,
    )
    return resp.choices[0].message.content

### Explicação: Função de Pipeline Multi-Etapas
Implementa o pipeline: planejar → analisar etapas → executar cada uma com roteamento baseado em intenção → refinar os resultados combinados. Retorna um dicionário estruturado para inspeção ou avaliação.


In [17]:
def pipeline(task: str):
    """Multi-step pipeline: plan → execute steps → refine final answer.

    Returns dict with keys: plan (raw plan text), steps (list of tuples), final (refined answer).
    Each step tuple: (index, step_text, step_result, model_alias_used)
    """
    # 1. Plan
    plan_alias = pick_model('general')
    plan_prompt = (
        "Break the task into 3 concise, actionable steps (no extra commentary).\n"
        f"Task: {task}"
    )
    plan = chat(plan_alias, plan_prompt)

    # 2. Parse steps (robust to numbering or bullet styles)
    raw_lines = [l.strip() for l in plan.splitlines() if l.strip()]
    steps = []
    for line in raw_lines:
        cleaned = re.sub(r'^\d+[).:-]?\s*', '', line)  # remove leading numbering
        cleaned = re.sub(r'^[-*]\s*', '', cleaned)      # remove bullet markers
        if cleaned:
            steps.append(cleaned)
        if len(steps) == 3:
            break
    if not steps:
        steps = [task]

    # 3. Execute steps
    outputs = []
    for idx, step in enumerate(steps, 1):
        intent = detect_intent(step)
        exec_alias = pick_model(intent)
        exec_prompt = (
            f"Execute step {idx} for the overall task.\n"
            f"Overall task: {task}\n"
            f"Step {idx}: {step}\n"
            "Return a concise result focusing only on the step objective."
        )
        result = chat(exec_alias, exec_prompt)
        outputs.append((idx, step, result, exec_alias))

    # 4. Refine final answer
    refine_alias = pick_model('summarize')
    combined = "\n\n".join(
        f"Step {idx} ({alias}) Output:\n{res}" for idx, _step, res, alias in outputs
    )
    refine_prompt = (
        "You are a senior assistant. Synthesize these step outputs into a cohesive final answer.\n"
        "Ensure clarity, avoid repetition, and highlight improvements or key insights if relevant.\n\n"
        f"Task: {task}\n\n"
        f"Step Outputs:\n{combined}"
    )
    final_ans = chat(refine_alias, refine_prompt)

    return {"plan": plan, "steps": outputs, "final": final_ans}

### Explicação: Executar Tarefa de Exemplo
Executa o pipeline completo em uma tarefa orientada à refatoração, ilustrando intenções mistas (código + resumo). O dicionário de resultados mostra o plano bruto, os resultados de cada etapa com os aliases dos modelos escolhidos e a resposta final sintetizada.


In [18]:
result = pipeline('Generate a refactored version of a slow Python loop and summarize performance gains.')
result

{'plan': '1. Profile the existing slow Python loop to identify bottlenecks.\n2. Refactor the loop using optimized techniques (e.g., list comprehensions, map, or itertools).\n3. Compare the performance of the refactored loop with the original using a benchmarking tool and summarize the gains.',
 'steps': [(1,
   'Profile the existing slow Python loop to identify bottlenecks.',
   "To execute step 1, you would use a profiling tool like `cProfile` in Python to analyze the performance of the existing slow loop. Here's an example of how you might do this:\n\n```python\nimport cProfile\n\ndef slow_loop():\n    # Example of a slow loop\n    result = []\n    for i in range(1000000):\n        result.append(i * i)\n    return result\n\n# Profile the slow loop\ncProfile.run('slow_loop()', 'profile_stats')\n\n# To see the results, you can use pstats module\nimport pstats\np = pstats.Stats('profile_stats')\np.sort_stats('cumulative').print_stats()\n```\n\nThis code will run the `slow_loop` function

### Explicação: Exibir Objeto de Resultado  
Mostra a saída estruturada do pipeline para uma inspeção rápida ou avaliação posterior (por exemplo, medir a qualidade da etapa ou a eficácia do refinamento).



---

**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos para garantir a precisão, é importante estar ciente de que traduções automatizadas podem conter erros ou imprecisões. O documento original em seu idioma nativo deve ser considerado a fonte oficial. Para informações críticas, recomenda-se a tradução profissional realizada por humanos. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações incorretas decorrentes do uso desta tradução.
